<h1> 1. 데이터 불러오기
    
<h3> kaggle에서 iris dataset을 다운받는다

In [1]:
import pandas as pd
import os  # 경로를 불러오는 패키지

In [3]:
os.chdir(r'C:\Users\hjb38\Documents\데이터 분석 과정\data')
# 앞에 r 붙여줘야함

In [4]:
iris = pd.read_csv("IRIS.csv")

In [5]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
iris['species'].value_counts()  # 꽃의 종류

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: species, dtype: int64

In [9]:
iris['id'] = range(len(iris))  # 각각의 객체를 구별하기 위해 id 라는 컬럼을 새로 추가
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,id
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,2
3,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5.0,3.6,1.4,0.2,Iris-setosa,4


In [10]:
iris = iris[['id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']]
# id 가 맨 앞으로 오게 순서 변경
iris.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
1,1,4.9,3.0,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa


<h1> 2. KNN 실습(범주형 데이터 분류)
    
<h3> 특징
<h4>  : 데이터가 많으면 느리다.
<h3> 파라미터
<h4>  n_neighbors : 가장 가까운 몇 개를 볼건지

1) Train(학습) 데이터 추출

In [13]:
train = iris.sample(100, replace = False, random_state = 2020)
# 전체 150개 데이터 중 100개의 데이터를 샘플로하여 비복원추출
# 나머지 50개는 테스트 데이터로 쓸 것임

train.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
104,104,6.5,3.0,5.8,2.2,Iris-virginica
8,8,4.4,2.9,1.4,0.2,Iris-setosa
61,61,5.9,3.0,4.2,1.5,Iris-versicolor
54,54,6.5,2.8,4.6,1.5,Iris-versicolor
78,78,6.0,2.9,4.5,1.5,Iris-versicolor


In [19]:
# index의 순서가 뒤직박죽일 때는 reset으로 컬럼으로 빼준 뒤 없앤다.
train = train.reset_index().drop(['index'], axis=1)
train.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,104,6.5,3.0,5.8,2.2,Iris-virginica
1,8,4.4,2.9,1.4,0.2,Iris-setosa
2,61,5.9,3.0,4.2,1.5,Iris-versicolor
3,54,6.5,2.8,4.6,1.5,Iris-versicolor
4,78,6.0,2.9,4.5,1.5,Iris-versicolor


2) Test(검증) 데이터 추출

In [24]:
~iris['id'].isin(train['id'])
# iris 의 id가 train 의id에 포함되는가? True면 train 용 데이터 False면 test 용 데이터
# test 용 데이터인 False 를 사용해야하기 때문에 ~ 를 이용해 test 용 데이터를 True로 바꿔준다.

0      False
1       True
2      False
3       True
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Name: id, Length: 150, dtype: bool

In [25]:
test = iris.loc[iris['id'].isin(train['id'])]
test.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa
5,5,5.4,3.9,1.7,0.4,Iris-setosa
7,7,5.0,3.4,1.5,0.2,Iris-setosa


In [28]:
# index가 또 뒤죽박죽 돼서 다시 reset
test.reset_index().drop(['index'], axis=1)
test.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa
5,5,5.4,3.9,1.7,0.4,Iris-setosa
7,7,5.0,3.4,1.5,0.2,Iris-setosa


3) train 데이터를 학습시키기 위해 패키지를 불러옴

In [31]:
from sklearn.neighbors import KNeighborsClassifier

4) 모델 정의하기

In [42]:
knn = KNeighborsClassifier(n_neighbors = 3)
# n_neighbors를 곧 K라고 생각하면 됨, 임의로 3이라고 지정

5) 모델 학습시키기

In [33]:
knn.fit(train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train['species'])
# 인풋변수들(sepal_length ~ )petal_width 및 타겟변수(species)를 지정한다.

KNeighborsClassifier(n_neighbors=3)

6) 학습된 모델을 바탕으로 test 데이터에 예측해보기

In [37]:
predictions = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

In [38]:
test['pred'] = predictions  # pred라는 컬럼을 추가해 실제 값인 species 와 비교한다
test.head()

<ipython-input-38-da656ba2e234>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions


,id,sepal_length,sepal_width,petal_length,petal_width,species,pred
0,0,5.1,3.5,1.4,0.2,Iris-setosa,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa,Iris-setosa
5,5,5.4,3.9,1.7,0.4,Iris-setosa,Iris-setosa
7,7,5.0,3.4,1.5,0.2,Iris-setosa,Iris-setosa


In [41]:
# 실제값인 species와  예측값인 pred가 얼마나 잘 맞는지 비교한다.
(test['species'] == test['pred']).mean()

# k가 3일때 98% 의 정확도가 나옴

0.98

<h1> 3. 최적의 K 찾기

In [48]:
# 위의 코드를 그대로 불러온 뒤 n_neighbors =  의 값을 반복문으로 바꿔준다.

for k in range(1,30):

    knn = KNeighborsClassifier(n_neighbors = k) 

    knn.fit(train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train['species'])

    predictions = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

    test['pred'] = predictions  

    print((test['species'] == test['pred']).mean())

# 만약 가장 높은 확률을 가진 k가 여러개라면 그 중 가장 작은 k를 선택해야 데이터 처리 시간을 단축할 수 있다.

<ipython-input-48-c791215f3faf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions
<ipython-input-48-c791215f3faf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions
<ipython-input-48-c791215f3faf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

1.0
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.97
0.98
0.98
0.99
0.99
0.99
0.97
0.98
0.98
0.99
0.99
0.98
0.97
0.97
0.97
0.97
0.96
0.98
0.97
0.98
0.97


<ipython-input-48-c791215f3faf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions


k가 12일 때 99%로 가장 최적이다.